In [1]:
import pandas as pd
data = pd.read_excel('./data/충원률/전체대학 충원율(2009~2021).xlsx')

In [2]:
df = data.groupby('연도').sum()

In [3]:
df.index = pd.to_datetime(df.index, format = '%Y')

In [8]:
df.columns

Index(['입학정원', '모집인원_계', '모집인원_정원내', '모집인원_정원외', '지원자_계', '지원자_정원내', '지원자_정원외',
       '입학자_계', '입학자_정원내_계', '입학자_정원내_남', '입학자_정원내_여', '입학자_정원외_계',
       '입학자_정원외_남', '입학자_정원외_여'],
      dtype='object')

In [65]:
df.shape

(13, 14)

In [29]:
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt
from statsmodels.tsa.holtwinters import ExponentialSmoothing as HWES

fit1 = SimpleExpSmoothing(df_test, initialization_method="estimated").fit()
fit2 = Holt(df_test, initialization_method="estimated").fit()
fit3 = Holt(df_test,  exponential=True, initialization_method="estimated").fit()
fit4 = Holt(df_test, damped_trend=True, initialization_method="estimated").fit()
fit5 = Holt(df_test, exponential=True, damped_trend=True, initialization_method="estimated").fit()
fit6 = HWES(df_test, seasonal_periods=2, trend='add', seasonal='add').fit(optimized=True, use_brute=True)
fit7 = HWES(df_test, seasonal_periods=2, trend='add', seasonal='mul').fit(optimized=True, use_brute=True)

C:\Python39\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  warnings.warn('No frequency information was'
C:\Python39\lib\site-packages\statsmodels\tsa\holtwinters\model.py:920: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
C:\Python39\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  warnings.warn('No frequency information was'
C:\Python39\lib\site-packages\statsmodels\tsa\holtwinters\model.py:920: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
C:\Python39\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  warnings.warn('No frequency information was'
C:\Python39\lib\site-packages\statsmodels\

In [30]:
forecast_1 = fit1.forecast(5)
forecast_2 = fit2.forecast(5)
forecast_3 = fit3.forecast(5)
forecast_4 = fit4.forecast(5)
forecast_5 = fit5.forecast(5)
forecast_6 = fit6.forecast(5)
forecast_7 = fit7.forecast(5)

In [31]:
t_p = pd.DataFrame({'f1': forecast_1, 'f2': forecast_2, 'f3': forecast_3, 'f4': forecast_4, 'f5': forecast_5, 'f6': forecast_6, 'f7': forecast_7})
t_p

,f1,f2,f3,f4,f5,f6,f7
2019-01-01,584985.955227,577823.851359,584047.232717,579662.839105,588462.571880,571940.814821,572116.311935
2020-01-01,584985.955227,567545.663479,575244.150210,570141.480187,585016.603067,559869.826187,559697.661085
2021-01-01,584985.955227,557267.475599,566573.752625,560715.235546,581624.971847,547851.452641,547977.296028
2022-01-01,584985.955227,546989.287718,558034.040061,551383.155033,578286.626320,535780.464007,535573.693859
2023-01-01,584985.955227,536711.099838,549623.042763,542144.297988,575000.539212,523762.090461,523838.280121


In [55]:
for i in range(1,8):
    print(((df['입학정원'][10:13]-globals()[f'forecast_{i}'][0:3])**2).mean())

108357535.78086789
80604890.22794177
8183770.119034618
40633717.62979547
92854641.98348343
275476659.7048511
274543233.2638713


In [56]:
model_mse = math.inf
model_num = 0
for i in range(1,8):    
    temp_model_mse = ((df['입학정원'][10:13]-globals()[f'forecast_{i}'][0:3])**2).mean()
    if temp_model_mse < model_mse :
        model_mse = temp_model_mse
        model_num = i
print(f'최적의 모델 : f{model_num}, 평가 점수 : {model_mse}')

최적의 모델 : f3, 평가 점수 : 8183770.119034618


In [109]:
df_time = df.copy()

In [77]:
model3 = Holt(df.iloc[:,0], exponential=True, initialization_method="estimated").fit()

C:\Python39\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  warnings.warn('No frequency information was'
C:\Python39\lib\site-packages\statsmodels\tsa\holtwinters\model.py:920: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(


In [82]:
df.iloc[:,0]

연도
2009-01-01    671884
2010-01-01    668364
2011-01-01    666668
2012-01-01    654841
2013-01-01    635645
2014-01-01    639199
2015-01-01    620727
2016-01-01    605806
2017-01-01    595472
2018-01-01    584933
2019-01-01    580478
2020-01-01    576295
2021-01-01    569846
Name: 입학정원, dtype: int64

In [90]:
import numpy as np
pre_data1 = pd.DataFrame({df.iloc[:,0].name : globals()['model3'].forecast(27)}).astype(np.int64)
pre_data1=pre_data1.rename(index='연도')

In [104]:
pre_data1.index.rename('연도',inplace=True)

In [105]:
pre_data1

,입학정원
연도,
2022-01-01,558563
2023-01-01,550244
2024-01-01,542048
2025-01-01,533974
2026-01-01,526021
2027-01-01,518186
2028-01-01,510467
2029-01-01,502864
2030-01-01,495374


In [97]:
pre_data1

,입학정원
2022-01-01,558563
2023-01-01,550244
2024-01-01,542048
2025-01-01,533974
2026-01-01,526021
2027-01-01,518186
2028-01-01,510467
2029-01-01,502864
2030-01-01,495374
2031-01-01,487995


In [5]:
df_time = df.copy()

In [6]:
import pandas as pd
import numpy as np
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt
from statsmodels.tsa.holtwinters import ExponentialSmoothing as HWES
import math
for j in range(df.shape[1]):
    df_test = df.iloc[:10,j]
    df_train = df.iloc[:,j]
    fit1 = SimpleExpSmoothing(df_test, initialization_method="estimated").fit()
    fit2 = Holt(df_test, initialization_method="estimated").fit()
    fit3 = Holt(df_test,  exponential=True, initialization_method="estimated").fit()
    fit4 = Holt(df_test, damped_trend=True, initialization_method="estimated").fit()
    fit5 = Holt(df_test, exponential=True, damped_trend=True, initialization_method="estimated").fit()
    fit6 = HWES(df_test, seasonal_periods=2, trend='add', seasonal='add').fit(optimized=True, use_brute=True)
    fit7 = HWES(df_test, seasonal_periods=2, trend='add', seasonal='mul').fit(optimized=True, use_brute=True)
    forecast_1 = fit1.forecast(5)
    forecast_2 = fit2.forecast(5)
    forecast_3 = fit3.forecast(5)
    forecast_4 = fit4.forecast(5)
    forecast_5 = fit5.forecast(5)
    forecast_6 = fit6.forecast(5)
    forecast_7 = fit7.forecast(5)
    model1 = SimpleExpSmoothing(df_train, initialization_method="estimated").fit()
    model2 = Holt(df_train, initialization_method="estimated").fit()
    model3 = Holt(df_train,  exponential=True, initialization_method="estimated").fit()
    model4 = Holt(df_train, damped_trend=True, initialization_method="estimated").fit()
    model5 = Holt(df_train, exponential=True, damped_trend=True, initialization_method="estimated").fit()
    model6 = HWES(df_train, seasonal_periods=2, trend='add', seasonal='add').fit(optimized=True, use_brute=True)
    model7 = HWES(df_train, seasonal_periods=2, trend='add', seasonal='mul').fit(optimized=True, use_brute=True)
    model_mse = math.inf
    model_num = 0
    for i in range(1,8):    
        temp_model_mse = ((df_train[10:13]-globals()[f'forecast_{i}'][0:3])**2).mean()
        if globals()[f'forecast_{i}'][0] == globals()[f'forecast_{i}'][1] and globals()[f'forecast_{i}'][1] == globals()[f'forecast_{i}'][2]:
            continue
        if (globals()[f'model{i}'].forecast(29)<0).sum() >0:
            continue
        if temp_model_mse < model_mse :
            model_mse = temp_model_mse
            model_num = i
    print(f'최적의 모델 : f{model_num}, 평가 점수 : {model_mse}')
    globals()[f'pre_data{j+1}'] = pd.DataFrame({df_train.name : globals()[f'model{model_num}'].forecast(29)}).astype(np.int64)
    globals()[f'pre_data{j+1}'].index.rename('연도',inplace=True)
    df_time = df_time.append(globals()[f'pre_data{j+1}'])

C:\Python39\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  warnings.warn('No frequency information was'
C:\Python39\lib\site-packages\statsmodels\tsa\holtwinters\model.py:920: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
C:\Python39\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  warnings.warn('No frequency information was'
C:\Python39\lib\site-packages\statsmodels\tsa\holtwinters\model.py:920: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
C:\Python39\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  warnings.warn('No frequency information was'
C:\Python39\lib\site-packages\statsmodels\

최적의 모델 : f3, 평가 점수 : 8183770.119034618


C:\Python39\lib\site-packages\statsmodels\tsa\holtwinters\model.py:920: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
C:\Python39\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  warnings.warn('No frequency information was'
C:\Python39\lib\site-packages\statsmodels\tsa\holtwinters\model.py:920: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
C:\Python39\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  warnings.warn('No frequency information was'
C:\Python39\lib\site-packages\statsmodels\tsa\holtwinters\model.py:920: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
C:\Python39\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequen

최적의 모델 : f3, 평가 점수 : 593142558.1963277


C:\Python39\lib\site-packages\statsmodels\tsa\holtwinters\model.py:920: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
C:\Python39\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  warnings.warn('No frequency information was'
C:\Python39\lib\site-packages\statsmodels\tsa\holtwinters\model.py:920: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
C:\Python39\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  warnings.warn('No frequency information was'
C:\Python39\lib\site-packages\statsmodels\tsa\holtwinters\model.py:920: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
C:\Python39\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequen

...

C:\Python39\lib\site-packages\statsmodels\tsa\holtwinters\model.py:920: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
C:\Python39\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  warnings.warn('No frequency information was'
C:\Python39\lib\site-packages\statsmodels\tsa\holtwinters\model.py:920: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
C:\Python39\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  warnings.warn('No frequency information was'
C:\Python39\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  warnings.warn('No frequency information was'
C:\Python39\lib\site-packages\statsmodels\

최적의 모델 : f5, 평가 점수 : 578959.1322601829


C:\Python39\lib\site-packages\statsmodels\tsa\holtwinters\model.py:920: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
C:\Python39\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  warnings.warn('No frequency information was'
C:\Python39\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  warnings.warn('No frequency information was'
C:\Python39\lib\site-packages\statsmodels\tsa\holtwinters\model.py:920: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
C:\Python39\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  warnings.warn('No frequency information was'
C:\Python39\lib\site-packages\statsmodels\

최적의 모델 : f3, 평가 점수 : 8984936.52659274


C:\Python39\lib\site-packages\statsmodels\tsa\holtwinters\model.py:920: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(


In [7]:
pre_data = pre_data1
for i in range(2,15):
    pre_data = pre_data.join(globals()[f'pre_data{i}'])
df_time = df.copy()
df_time = df_time.append(pre_data)

In [8]:
df.columns

Index(['입학정원', '모집인원_계', '모집인원_정원내', '모집인원_정원외', '지원자_계', '지원자_정원내', '지원자_정원외',
       '입학자_계', '입학자_정원내_계', '입학자_정원내_남', '입학자_정원내_여', '입학자_정원외_계',
       '입학자_정원외_남', '입학자_정원외_여'],
      dtype='object')

In [8]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

rf1 = RandomForestRegressor()
X = df_time.iloc[:,[0,2,3,5,6]]
y = df_time.iloc[:,8]
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=70)
rf1.fit(X_train, y_train)
print(rf1.score(X_train, y_train))
print(rf1.score(X_test, y_test))

0.9983830634163017
0.994800005094939


In [21]:
print(X_test,"\n",rf1.predict(X_test))

              입학정원  모집인원_정원내  모집인원_정원외  지원자_정원내  지원자_정원외
연도                                                      
2026-01-01  526021    523694    226732  4559073   286194
2040-01-01  426343    413128    495966  4467543   272167
2015-01-01  620727    620937    131355  4636239   269987
2044-01-01  401503    384291    608290  4441391   268623
2043-01-01  407574    391392    578468  4447929   269492
2021-01-01  569846    570402    168116  3625048   290381
2038-01-01  439333    427986    446445  4480619   274012
2014-01-01  639199    639927    136325  4523592   267761
2037-01-01  445976    435529    423233  4487157   274953 
 [441804.81 284658.68 580041.65 238031.4  262224.8  498764.99 311381.38
 594337.29 333375.37]


In [111]:
df_time.columns

Index(['입학정원', '모집인원_계', '모집인원_정원내', '모집인원_정원외', '지원자_계', '지원자_정원내', '지원자_정원외',
       '입학자_계', '입학자_정원내_계', '입학자_정원내_남', '입학자_정원내_여', '입학자_정원외_계',
       '입학자_정원외_남', '입학자_정원외_여'],
      dtype='object')

In [51]:
fit1 = SimpleExpSmoothing(df_test, initialization_method="estimated").fit()
fit2 = Holt(df_test, initialization_method="estimated").fit()
fit3 = Holt(df_test,  exponential=True, initialization_method="estimated").fit()
fit4 = Holt(df_test, damped_trend=True, initialization_method="estimated").fit()
fit5 = Holt(df_test, exponential=True, damped_trend=True, initialization_method="estimated").fit()
fit6 = HWES(df_test, seasonal_periods=2, trend='add', seasonal='add').fit(optimized=True, use_brute=True)
fit7 = HWES(df_test, seasonal_periods=2, trend='add', seasonal='mul').fit(optimized=True, use_brute=True)
forecast_1 = fit1.forecast(5)
forecast_2 = fit2.forecast(5)
forecast_3 = fit3.forecast(5)
forecast_4 = fit4.forecast(5)
forecast_5 = fit5.forecast(5)
forecast_6 = fit6.forecast(5)
forecast_7 = fit7.forecast(5)
t_p = pd.DataFrame({'f1': forecast_1, 'f2': forecast_2, 'f3': forecast_3, 'f4': forecast_4, 'f5': forecast_5, 'f6': forecast_6, 'f7': forecast_7})
t_p

C:\Python39\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  warnings.warn('No frequency information was'
C:\Python39\lib\site-packages\statsmodels\tsa\holtwinters\model.py:920: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
C:\Python39\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  warnings.warn('No frequency information was'
C:\Python39\lib\site-packages\statsmodels\tsa\holtwinters\model.py:920: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
C:\Python39\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  warnings.warn('No frequency information was'
C:\Python39\lib\site-packages\statsmodels\

,f1,f2,f3,f4,f5,f6,f7
2022-01-01,4.682352e+06,4.831626e+06,4.833821e+06,4.842546e+06,5.499989e+06,4.046844e+06,4.017596e+06
2023-01-01,4.682352e+06,4.820528e+06,4.823077e+06,4.833002e+06,5.537233e+06,3.775212e+06,3.779039e+06
2024-01-01,4.682352e+06,4.809431e+06,4.812357e+06,4.823553e+06,5.574354e+06,3.579415e+06,3.539460e+06
2025-01-01,4.682352e+06,4.798333e+06,4.801661e+06,4.814198e+06,5.611348e+06,3.307783e+06,3.300838e+06
2026-01-01,4.682352e+06,4.787236e+06,4.790988e+06,4.804937e+06,5.648214e+06,3.111987e+06,3.061324e+06


In [59]:
if fit6.forecast(50) in 

AttributeError: 'Series' object has no attribute 'type'

In [79]:
(fit6.forecast(50) < 0)

AttributeError: 'numpy.ndarray' object has no attribute 'count'

In [47]:
df.iloc[:,1]

연도
2009-01-01    803936
2010-01-01    809549
2011-01-01    807959
2012-01-01    796666
2013-01-01    760491
2014-01-01    776252
2015-01-01    752292
2016-01-01    744338
2017-01-01    732459
2018-01-01    727953
2019-01-01    723329
2020-01-01    738849
2021-01-01    738518
Name: 모집인원_계, dtype: int64

In [39]:
pre_data

,입학정원,모집인원_계,모집인원_정원내,모집인원_정원외,지원자_계,지원자_정원내,지원자_정원외,입학자_계,입학자_정원내_계,입학자_정원내_남,입학자_정원내_여,입학자_정원외_계,입학자_정원외_남,입학자_정원외_여
연도,,,,,,,,,,,,,,
2022-01-01,558563,738168,558248,177530,4046843,3767771,290725,560170,488762,235384,240888,79992,37495,49244
2023-01-01,550244,738168,549479,188901,3775211,3466737,289568,548956,477765,230505,231041,80487,37428,53664
2024-01-01,542048,738168,540797,200875,3579415,3296497,288428,537743,466768,225727,224853,80979,37361,58482
2025-01-01,533974,738168,532202,213477,3307783,2995463,287303,526530,455771,221049,215132,81470,37295,63732
2026-01-01,526021,738168,523694,226732,3111986,2825222,286194,515317,444774,216467,208817,81958,37231,69453
2027-01-01,518186,738168,515270,240664,2840355,2524188,285101,504103,433777,211980,199223,82445,37166,75687
2028-01-01,510467,738168,506930,255301,2644558,2353948,284022,492890,422780,207586,192782,82929,37103,82482
2029-01-01,502864,738168,498674,270668,2372926,2052913,282958,481677,411783,203284,183314,83412,37040,89886
2030-01-01,495374,738168,490500,286792,2177130,1882673,281909,470464,400786,199070,176747,83892,36978,97955


In [40]:
df_time = df.copy()
df_time = df_time.append(pre_data)

In [41]:
df_time

,입학정원,모집인원_계,모집인원_정원내,모집인원_정원외,지원자_계,지원자_정원내,지원자_정원외,입학자_계,입학자_정원내_계,입학자_정원내_남,입학자_정원내_여,입학자_정원외_계,입학자_정원외_남,입학자_정원외_여
연도,,,,,,,,,,,,,,
2009-01-01,671884,803936,669315,134621,4182669,3920582,262087,682119,612097,313566,298531,70022,35864,34158
2010-01-01,668364,809549,668745,140804,4845518,4526486,319032,698628,620321,312307,308014,78307,39602,38705
2011-01-01,666668,807959,668278,139681,5695255,5364396,330859,699852,623627,312863,310764,76225,39770,36455
2012-01-01,654841,796666,658045,138621,5706011,5370730,335281,684100,606613,307888,298725,77487,40331,37156
2013-01-01,635645,760491,635016,125475,4803151,4545963,257188,656241,588681,297623,291058,67560,33712,33848
2014-01-01,639199,776252,639927,136325,4791353,4523592,267761,657957,585609,294221,291388,72348,35697,36651
2015-01-01,620727,752292,620937,131355,4906226,4636239,269987,639730,569540,283023,286517,70190,34021,36169
2016-01-01,605806,744338,606682,137656,4839424,4581696,257728,624446,552228,270541,281687,72218,34783,37435
2017-01-01,595472,732459,595056,137403,4758971,4496320,262651,606784,535412,263370,272042,71372,34127,37245


In [11]:
df_time = df.copy()

In [13]:
df_time

,입학정원,모집인원_계,모집인원_정원내,모집인원_정원외,지원자_계,지원자_정원내,지원자_정원외,입학자_계,입학자_정원내_계,입학자_정원내_남,입학자_정원내_여,입학자_정원외_계,입학자_정원외_남,입학자_정원외_여
연도,,,,,,,,,,,,,,
2009-01-01,671884.0,803936.0,669315.0,134621.0,4182669.0,3920582.0,262087.0,682119.0,612097.0,313566.0,298531.0,70022.0,35864.0,34158.0
2010-01-01,668364.0,809549.0,668745.0,140804.0,4845518.0,4526486.0,319032.0,698628.0,620321.0,312307.0,308014.0,78307.0,39602.0,38705.0
2011-01-01,666668.0,807959.0,668278.0,139681.0,5695255.0,5364396.0,330859.0,699852.0,623627.0,312863.0,310764.0,76225.0,39770.0,36455.0
2012-01-01,654841.0,796666.0,658045.0,138621.0,5706011.0,5370730.0,335281.0,684100.0,606613.0,307888.0,298725.0,77487.0,40331.0,37156.0
2013-01-01,635645.0,760491.0,635016.0,125475.0,4803151.0,4545963.0,257188.0,656241.0,588681.0,297623.0,291058.0,67560.0,33712.0,33848.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2046-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,387590.0
2047-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,422383.0
2048-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,460300.0
